In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import json
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from time import sleep
import os
from collections import Counter
import pickle
import warnings
import time
warnings.filterwarnings("ignore")

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import PIL
from PIL import Image, ImageFilter
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

import boto3
import botocore
%matplotlib inline

# Use proxy and headers for safe web scraping
# os.environ['HTTPS_PROXY'] = 'http://3.112.188.39:8080'

# pd.options.mode.chained_assignment = None
headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/'
    '537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}

In [19]:
link = 'https://www.amazon.co.uk/Olaplex-No-5-Bond-Maintenance-Conditioner/dp/B07D37SBHF/ref=lp_2867976031_1_4_s_it?s=beauty&ie=UTF8&qid=1585929007&sr=1-4'

def browser(link):
    '''This funtion opens a selenium based chromebrowser specifically tuned 
    to work for amazon product(singular item) webpages. Few functionality 
    includes translation of webpage, clicking the initial popups, and hovering
    over product imagesso that the images can be scrape
    
    PARAMETER
    ---------
    link: str
        Amazon Product item link
        
    RETURN
    ------
    driver: Selenium web browser with operated functions
    '''
    options = Options()
    prefs = {
      "translate_whitelists": {"ja":"en","de":'en'},
      "translate":{"enabled":"true"}
    }
#     helium = r'C:\Users\Dell-pc\AppData\Local\Google\Chrome\User Data\Default\Extensions\njmehopjdpcckochcggncklnlmikcbnb\4.2.12_0'
#     options.add_argument(helium)
    options.add_experimental_option("prefs", prefs)
    options.headless = True
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(link)
    try:
        driver.find_element_by_xpath('//*[@id="nav-main"]/div[1]/div[2]/div/div[3]/span[1]/span/input').click()
    except:
        pass
    try:
        hover(driver,'//*[@id="altImages"]/ul/li[3]')
    except:
        pass
    try:
        driver.find_element_by_xpath('//*[@id="a-popover-6"]/div/header/button/i').click()
    except:
        pass
    try:
        hover(driver,'//*[@id="altImages"]/ul/li[4]')
    except:
        pass
    try:
        driver.find_element_by_xpath('//*[@id="a-popover-6"]/div/header/button/i').click()
    except:
        pass
    try:
        hover(driver,'//*[@id="altImages"]/ul/li[5]')
    except:
        pass  
    try:
        driver.find_element_by_xpath('//*[@id="a-popover-6"]/div/header/button/i').click()
    except:
        pass
    try:
        hover(driver,'//*[@id="altImages"]/ul/li[6]')
    except:
        pass
    try:
        driver.find_element_by_xpath('//*[@id="a-popover-6"]/div/header/button/i').click()
    except:
        pass
    try:    
        hover(driver,'//*[@id="altImages"]/ul/li[7]')
    except:
        pass
    try:
        driver.find_element_by_xpath('//*[@id="a-popover-6"]/div/header/button/i').click()
    except:
        pass
    try:
        hover(driver,'//*[@id="altImages"]/ul/li[8]')
    except:
        pass
    try:
        driver.find_element_by_xpath('//*[@id="a-popover-6"]/div/header/button/i').click()
    except:
        pass
    try:
        hover(driver,'//*[@id="altImages"]/ul/li[9]')
    except:
        pass
    try:
        driver.find_element_by_xpath('//*[@id="a-popover-6"]/div/header/button/i').click()
    except:
        pass
    return driver

def scroll(driver):
    scroll_temp(driver)
    from selenium.common.exceptions import NoSuchElementException
    try:
        try:
            element = driver.find_element_by_xpath('//*[@id="reviewsMedley"]/div/div[1]')
        except NoSuchElementException:
            try:
                element = driver.find_element_by_xpath('//*[@id="reviewsMedley"]') 
            except NoSuchElementException:
                element = driver.find_element_by_xpath('//*[@id="detail-bullets_feature_div"]')
        actions = ActionChains(driver)
        actions.move_to_element(element).perform()
    except NoSuchElementException:
        pass
    
def scroll_temp(driver):
    ''' 
    Automated Scroller in Selenium Webbrowser
    
    PARAMETER
    ---------
    driver: Selenium Webbrowser
    '''
    pre_scroll_height = driver.execute_script('return document.body.scrollHeight;')
    run_time, max_run_time = 0, 2
    while True:
        iteration_start = time.time()
        # Scroll webpage, the 100 allows for a more 'aggressive' scroll
        driver.execute_script('window.scrollTo(0,0.6*document.body.scrollHeight);')

        post_scroll_height = driver.execute_script('return document.body.scrollHeight;')

        scrolled = post_scroll_height != pre_scroll_height
        timed_out = run_time >= max_run_time

        if scrolled:
            run_time = 0
            pre_scroll_height = post_scroll_height
        elif not scrolled and not timed_out:
            run_time += time.time() - iteration_start
        elif not scrolled and timed_out:
            break
    

#Opening Selenium Webdrive with Amazon product
driver = browser(link)
time.sleep(5)
# scroll(driver)
# time.sleep(2)

#Initializing BeautifulSoup operation in selenium browser
import requests
req = requests.get(link, headers=headers)
soup = BeautifulSoup(req.content, 'html.parser')
time.sleep(2)

#Product Title
try:
#     product_title = driver.find_element_by_xpath('//*[@id="productTitle"]').text
    product_title = soup
except:
    product_title = 'Not Scrapable'

print(product_title)

# #Ratings - Star
# try:
#     rating_star = float(selenium_soup.findAll('span',{'class':'a-icon-alt'})[0].text.split()[0])
# except:
#     rating_star = 'Not Scrapable'
# print(rating_star)


<!--
        To discuss automated access to Amazon data please contact api-services-support@amazon.com.
        For information about migrating to our APIs refer to our Marketplace APIs at https://developer.amazonservices.co.uk/ref=rm_5_sv, or our Product Advertising API at https://affiliate-program.amazon.co.uk/gp/advertising/api/detail/main.html/ref=rm_5_ac for advertising use cases.
-->
<html>
<head>
<title>503 - Service Unavailable Error</title>
</head>
<body bgcolor="#FFFFFF" text="#000000">
<center>
<a href="https://www.amazon.co.uk/ref=cs_503_logo/">
<img alt="Amazon.co.uk" border="0" height="45" src="https://images-eu.ssl-images-amazon.com/images/G/02/uk-shared/logos/smile-amazon-logo-200x45.gif" width="200"/></a>
<p align="center">
<font face="Verdana,Arial,Helvetica">
<font color="#CC6600" size="+2"><b>We're sorry</b></font><br/>
<b>An error occurred when we tried to process your request.<br/>We're working on the problem and expect to resolve it shortly. Please note that if y

In [ ]:
# //*[@id="customer_review-R3QCGHM0OXNSVI"]/div[2]/a[2]/span

In [ ]:
# //*[@id="productTitle"]